In [1]:
import pandas as pd
import numpy as np
import time
import csv
from copy import deepcopy
import matplotlib.pyplot as plt
import time

Importing and cleaning data

In [2]:
#appending data in dictionaru with unique ID number as key and attribute list as dictionary
data={}
i=-1
with open('diabetes1.arff') as a:
    r=csv.reader(a)
    for line in r:
        try:
            x=int(line[0])
            i+=1
            data[i]=[]
            for num in line:
                if num!='tested_positive' or num!='tested_negative':
                    data[i].append(float(num))
        except:
            i=i

Creating dataframe in pandas

In [3]:
df_data=[]
for point in data:
    df_data.append(data[point])
col=['att1','att2','att3','att4','att5','att6','att7','att8']

In [4]:
df=pd.DataFrame(df_data,columns =col)

Standardising data

In [5]:
df1=((df-df.mean())/df.std())

In [6]:
data1=np.array(df1)

Creating distance matrix for reference

In [7]:
distance=np.ones((len(data1),len(data1)))

Creating function to calculate eucledian distance between two points

In [8]:
def d_cal(a,b):
    global data1
    a_val=data1[a]
    b_val=data1[b]
    out=[]
    for i in range(len(data[a])):
        y=(a_val[i]-b_val[i])**2
        out.append(y)
    
    return (sum(out)**0.5)

In [9]:
for i in range(len(data1)):
    for j in range(len(data1)):
        distance[i][j]=d_cal(i,j)

In [10]:
i=0
data={}
for row in data1:
    data[i]=list(row)
    i+=1

Creating object for each data point

In [11]:
class d_point:
    #unique id argument to create object
    def __init__(self,id):
        self.id=id
        #individual cluster contain point itselt before iteration
        self.cluster=[id]
        #this is to be updated later as peer result
        self.core=False
        self.border=False
        self.outlier=False

In [12]:
#creating objects using above class
#creating object list of points
d_obj=[]
for point in data:
    obj=d_point(point)
    d_obj.append(obj)

In [13]:
#creating function to find point in object list
def obj_finder(id):
    global d_obj
    for obj in d_obj:
        if obj.id==id:
            return obj
    

In [14]:
#this functions checks for each point in given eps radius
def cluster_maker(eps):
    global distance
    for row in range(distance.shape[0]):
        obj=obj_finder(row)
        for col in range(len(range(distance.shape[0]))):
            obj2=obj_finder(col)
            #if distace is less than or equal eps then added to cluster list 
            if distance[row][col]<=eps:
                if row not in obj2.cluster:
                    obj2.cluster.append(row)
            obj2.cluster.sort()
    return d_obj
                


In [15]:
def combine(temp,ob_list,ite,border):
    for i in range(ite):
        for obj1 in ob_list:
            if obj1.id not in border:
                if temp.intersection(set(obj1.cluster)):
                    temp=temp.union(set(obj1.cluster))
                    obj1.cluster=[]
        return list(temp)

def iterator(eps,k):
    #eps:radius of circle
    #k: number of minimum points required in that radius
    #getting circle data for each point(number point wuithin radius for every point)
    out2=cluster_maker(eps)
    out=deepcopy(out2)
    #creating result list which will contain each cluster list
    result=[]
    #list for all core points
    core=[]
    #list for suspected border points
    border=[]
    
   
    #if point contains points less than minimum number of pointn within radius then it may be border or noise point
    for obj in out:
        if len(obj.cluster)<k:
                border.append(obj.id)
    #if number point is more than or qual to minimum points then it is core point
    for obj in out:
        #appending that id in core list
        if len(obj.cluster)>=k:
            core.append(obj.id)
            
    #removing border points to avoid it to co be conder as core point
    for obj in out:
        for point in obj.cluster:
            if point in border:
                obj.cluster.remove(point)
                
    #creating clusters by combining clusters by set operation
    for obj in out:
        if obj.id not in border:
            temp=set(obj.cluster).intersection(core)
            x=combine(temp,out,5,border)
            #print(x)
            if x not in result:
                result.append(x)
  
    #again performing set operation on lists in result to avoid repetation of points 
    result1=[]
    for j in range(len(result)):
        o=[]
        o.extend(result[j])
        for i in range(2):
            for i in range(len(result)):
                if set(o).intersection(set(result[i])):
                    o=list(set(o).union(set(result[i])))
                    
        if o not in result1:
            result1.append(o)
            
    result=result1
    return result,out2,border,core
    


In [16]:
result2,obj_list2,border,core=iterator(2,5)

In [17]:
border2=deepcopy(border)
#it is list of suspspected border points
#thare are outliers and border points in this list


In [18]:
result=deepcopy(result2)
#defining function for detrmining border points
def min_for_border(eps):
    global obj_list
    global border2
    global core
    global distance
    global result
    outlier=[]
    #print(border2)
    for point in border2:
        temp=999999
        p=100000
        #checking each point in border point in cluster to check if any core point is present
        for point1 in core:
            #if there are multiple core points from different clusters then border point is assigned to nearest point cluster
            if distance[point][point1]<temp:
                temp=distance[point][point1]
                p=point1
                
        if temp<eps:
            #updating cluster in result with border point 
            for l in result:
                if p in l:
                    if point not in l:
                        l.append(point)
                       
        else:
            #if above is not satisfied then point is appebnded in outlier list
            outlier.append(point)
            
    noise_and_border=[]
    border_points=[]
    #differentiating border point from suspected border points
    for point in outlier:
        if point not in border2:
            border_points.append(point)
        else:
            border2.remove(point)
            
    noise_and_border.append(outlier)
    noise_and_border.append(border2)
    #returning lists of clusters, border points and noise points
    return result,noise_and_border
    #print(outlier)
            
            
            

In [19]:
result,noise_and_border=min_for_border(2)
noise_points=noise_and_border[0]
border_points=noise_and_border[1]

In [20]:
#reult
result.remove([])
i=0
for l in result:
    if len(l)!=0:
        i+=1
        print(f'cluster {i} population: {len(l)} ')
print()
print(f'Total number of noise points: {len(noise_points)}')
print(f'Total number of border points: {len(border_points)}')

cluster 1 population: 702 
cluster 2 population: 24 
cluster 3 population: 7 

Total number of noise points: 35
Total number of border points: 48


In [23]:
#dispalying all border points in cluster 2
print('IDs of border points in cluster 2')
for point in result[1]:
    if point in border2:
        print(point)
     

IDs of border points in cluster 2
332
336
604


In [24]:
#displaying all core points in cluster 2
print('IDs of Core points in cluster 2')
for point in result[1]:
    if point in core:
        print(point)

IDs of Core points in cluster 2
643
261
7
266
269
15
533
535
300
430
435
697
703
589
78
468
601
347
222
484
619
